# ChronoNet

Here we train the model and record the train and validation accuracy. We used subset of the temple dataset for faster training. we used 500 subjects only for training and validation of the model. The recorded metrics are accuracy, binary cross entropy loss, F1 score, Precision and Recall.

- Train Data: 500 subjects

- Test Data: 276 subjects

In [1]:
import torch
from tqdm import tqdm
import torch.nn as nn

from load_data import read_data_arrays, data_file_names, standardize_data, data_loader
from models import ChronoNet
from utils import cal_accuracy, evaluate_model 


BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 5

print("Reading Data....")
sample_size = 500
data_files = data_file_names(sample_size)
(train_features, val_features, test_features,
 train_labels, val_labels, test_labels, test_lengths) = read_data_arrays(
    data_files)
    
print("Scaling Data....")
train_features, val_features, test_features = standardize_data(
    train_features, val_features, test_features)
    
print("Data Loader....")
train_iter = data_loader(train_features, train_labels, DEVICE, BATCH_SIZE)
val_iter = data_loader(val_features, val_labels, DEVICE, BATCH_SIZE)
test_iter = data_loader(test_features, test_labels, DEVICE, BATCH_SIZE, shuffle=False)
    
print("Training Model....")
n_chans = 19
model=ChronoNet(n_chans)
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, val_iter)
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        cal_accuracy(model, train_iter)[0], ",F1: ", 
        cal_accuracy(model, train_iter)[4], ",Precision: ", 
        cal_accuracy(model, train_iter)[2], ",Recall: ", 
        cal_accuracy(model, train_iter)[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        cal_accuracy(model, val_iter)[0], ",F1: ", 
        cal_accuracy(model, val_iter)[4], ",Precision: ", 
        cal_accuracy(model, val_iter)[2], ",Recall: ", 
        cal_accuracy(model, val_iter)[3])

/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading Data....
Scaling Data....
Data Loader....
Training Model....
Epoch 1


100%|█████████████████████████████████████████| 692/692 [04:37<00:00,  2.49it/s]


Train loss: 0.6185583529100253 ,Train Accuracy:  0.7934802259887006 ,F1:  0.7612441378949981 ,Precision:  0.9020463762731804 ,Recall:  0.6584632768361582
Val loss: 0.6317908183320776 , Val Accuracy:  0.6661016949152543 ,F1:  0.5159229408295657 ,Precision:  0.9376563058235085 ,Recall:  0.355864406779661
Epoch 2


100%|█████████████████████████████████████████| 692/692 [03:47<00:00,  3.04it/s]


...

Train loss: 0.5769695649946356 ,Train Accuracy:  0.8252994350282485 ,F1:  0.7993771491598001 ,Precision:  0.9386561023921987 ,Recall:  0.696090395480226
Val loss: 0.611729929715524 , Val Accuracy:  0.7238305084745763 ,F1:  0.6365055994289028 ,Precision:  0.9308364870155291 ,Recall:  0.48359322033898305
Epoch 4


100%|█████████████████████████████████████████| 692/692 [03:44<00:00,  3.08it/s]


Train loss: 0.5716010270091151 ,Train Accuracy:  0.8470621468926554 ,F1:  0.8297291517278687 ,Precision:  0.935785023126472 ,Recall:  0.7452655367231639
Val loss: 0.607604176967175 , Val Accuracy:  0.7368135593220339 ,F1:  0.6608125819134995 ,Precision:  0.9291154791154791 ,Recall:  0.5127457627118645
Epoch 5


100%|█████████████████████████████████████████| 692/692 [03:53<00:00,  2.97it/s]


Train loss: 0.5678945554990988 ,Train Accuracy:  0.8633559322033898 ,F1:  0.8515376588300289 ,Precision:  0.9321596559602204 ,Recall:  0.7837514124293785
Val loss: 0.6037838544164386 , Val Accuracy:  0.7559661016949153 ,F1:  0.7058150463814311 ,Precision:  0.888385968521757 ,Recall:  0.5854915254237288


## Testing the model

In [2]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)
        
from collections import Counter

y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

print("Test Accuracy: ", accuracy_score(y_final, yhat_final))
print("Test Confusion: ",confusion_matrix(y_final, yhat_final))
print("Test F1: ", f1_score(y_final, yhat_final))
print("Test Recall: ", recall_score(y_final, yhat_final))
print("Test Precision: ", precision_score(y_final, yhat_final))

Test Accuracy:  0.8043478260869565
Test Confusion:  [[141   9]
 [ 45  81]]
Test F1:  0.75
Test Recall:  0.6428571428571429
Test Precision:  0.9


# LSTM

In [5]:
from models import LSTM

learning_rate = 5e-3

model = LSTM(input_size=500, num_channels=19, hidden_units=128)

print("Training Model....")
n_chans = 19
DEVICE = torch.device("cpu")
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
NUM_EPOCHS = 5
BATCH_SIZE = 128

print("started training")

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, test_iter)
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        cal_accuracy(model, train_iter)[0], ",F1: ", 
        cal_accuracy(model, train_iter)[4], ",Precision: ", 
        cal_accuracy(model, train_iter)[2], ",Recall: ", 
        cal_accuracy(model, train_iter)[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        cal_accuracy(model, val_iter)[0], ",F1: ", 
        cal_accuracy(model, val_iter)[4], ",Precision: ", 
        cal_accuracy(model, val_iter)[2], ",Recall: ", 
        cal_accuracy(model, val_iter)[3])

Training Model....
started training
Epoch 1


100%|█████████████████████████████████████████| 692/692 [08:51<00:00,  1.30it/s]


Train loss: 0.661770528279288 ,Train Accuracy:  0.7018870056497175 ,F1:  0.624489389259739 ,Precision:  0.843477257872275 ,Recall:  0.49577401129943505
Val loss: 0.67322963032068 , Val Accuracy:  0.5675593220338983 ,F1:  0.2789803877239586 ,Precision:  0.8386000679578661 ,Recall:  0.16732203389830508
Epoch 2


100%|█████████████████████████████████████████| 692/692 [36:10<00:00,  3.14s/it]


Train loss: 0.6248360861071273 ,Train Accuracy:  0.7378757062146892 ,F1:  0.6679738936279842 ,Precision:  0.9108829729096729 ,Recall:  0.5273446327683616
Val loss: 0.6709002801016265 , Val Accuracy:  0.5743050847457627 ,F1:  0.2901074053137365 ,Precision:  0.8727891156462585 ,Recall:  0.17396610169491525
Epoch 3


...

Train loss: 0.6036012842820558 ,Train Accuracy:  0.7931073446327683 ,F1:  0.7555798803929944 ,Precision:  0.9229991520448764 ,Recall:  0.6395706214689265
Val loss: 0.6608413919514301 , Val Accuracy:  0.602 ,F1:  0.3752461022721226 ,Precision:  0.8721246599060104 ,Recall:  0.2390508474576271
Epoch 4


100%|█████████████████████████████████████████| 692/692 [13:37<00:00,  1.18s/it]


Train loss: 0.5900325086075446 ,Train Accuracy:  0.8101694915254237 ,F1:  0.777824798984342 ,Precision:  0.937575718931327 ,Recall:  0.6645875706214689
Val loss: 0.6607672583823111 , Val Accuracy:  0.604 ,F1:  0.37907940895078135 ,Precision:  0.8774606299212598 ,Recall:  0.2417627118644068
Epoch 5


100%|█████████████████████████████████████████| 692/692 [17:01<00:00,  1.48s/it]


Train loss: 0.5812325871231928 ,Train Accuracy:  0.833683615819209 ,F1:  0.8115485564304462 ,Precision:  0.9361394181066313 ,Recall:  0.716225988700565
Val loss: 0.6539293941329507 , Val Accuracy:  0.6249830508474576 ,F1:  0.44049967126890205 ,Precision:  0.8670117459685447 ,Recall:  0.2952542372881356


In [6]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)

y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
print("Test Accuracy: ", accuracy_score(y_final, yhat_final))
print("Test Confusion: ",confusion_matrix(y_final, yhat_final))
print("Test F1: ", f1_score(y_final, yhat_final))
print("Test Recall: ", recall_score(y_final, yhat_final))
print("Test Precision: ", precision_score(y_final, yhat_final))

Test Accuracy:  0.6485507246376812
Test Confusion:  [[149   1]
 [ 96  30]]
Test F1:  0.38216560509554137
Test Recall:  0.23809523809523808
Test Precision:  0.967741935483871
